In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium



UsageError: Line magic function `%matplotlib inline` not found.


In [8]:
# Load datasets
ev_ownership = pd.read_csv('ev_ownership.csv')
charging_stations = pd.read_csv('charging_stations.csv')
population_density = pd.read_csv('population_density.csv')
traffic_congestion = pd.read_csv('traffic_congestion.csv')

# Inspect the first few rows
print("EV Ownership:")
print(ev_ownership.head())

print("\nCharging Stations:")
print(charging_stations.head())

print("\nPopulation Density:")
print(population_density.head())

print("\nTraffic Congestion:")
print(traffic_congestion.head())

EV Ownership:
      Clean Alternative Fuel Vehicle Type  VIN (1-10)  DOL Vehicle ID  \
0  Plug-in Hybrid Electric Vehicle (PHEV)  WBY8P8C51K       477294338   
1          Battery Electric Vehicle (BEV)  3FMTK4SE2P       233490204   
2          Battery Electric Vehicle (BEV)  5YJ3E1EAXK       478657924   
3  Plug-in Hybrid Electric Vehicle (PHEV)  WBY8P8C51K       477294338   
4          Battery Electric Vehicle (BEV)  3FMTK4SE2P       233490204   

   Model Year   Make           Model Primary Use  Electric Range  \
0        2019    BMW              i3   Passenger             126   
1        2023   FORD  Mustang Mach-E   Passenger               0   
2        2019  TESLA         Model 3   Passenger             220   
3        2019    BMW              i3   Passenger             126   
4        2023   FORD  Mustang Mach-E   Passenger               0   

   Odometer Reading                       Odometer Reading Description  ...  \
0                14                                     Act

In [9]:
# Check for missing values
print("Missing values in EV Ownership:", ev_ownership.isnull().sum())
print("Missing values in Charging Stations:", charging_stations.isnull().sum())
print("Missing values in Population Density:", population_density.isnull().sum())
print("Missing values in Traffic Congestion:", traffic_congestion.isnull().sum())

# Check data types
print("\nData types in EV Ownership:")
print(ev_ownership.dtypes)

Missing values in EV Ownership: Clean Alternative Fuel Vehicle Type                                     0
VIN (1-10)                                                              0
DOL Vehicle ID                                                          0
Model Year                                                              0
Make                                                                    0
Model                                                                   0
Primary Use                                                             0
Electric Range                                                          0
Odometer Reading                                                        0
Odometer Reading Description                                            0
New or Used Vehicle                                                     0
Sale Price                                                              0
Sale Date                                                          886677
Base M

#Step 1: Data Cleaning
In this step, we clean the dataset to handle missing values and ensure consistency in the data.

In [10]:
ev_ownership = ev_ownership.drop(columns=['UnnecessaryColumn'], errors='ignore')

In [18]:
# Split 'New Georeferenced Column' into Latitude and Longitude
charging_stations[['Latitude', 'Longitude']] = charging_stations['New Georeferenced Column'].str.split(',', expand=True)

# Convert to numeric
charging_stations['Latitude'] = charging_stations['Latitude'].astype(float)
charging_stations['Longitude'] = charging_stations['Longitude'].astype(float)

# Check the result
print(charging_stations[['Latitude', 'Longitude']].head())

    Latitude   Longitude
0  47.044011 -122.822404
1  47.443377 -122.296229
2  47.673347 -117.388933
3  47.655792 -117.423664
4  47.545324 -122.019500


## Step 2: Visualizing Existing EV Charging Infrastructure
We visualize the locations of existing EV charging stations to identify areas with high coverage and underserved regions. This provides a baseline understanding of the current infrastructure.

In [20]:
import folium

# Create a map centered in Washington State
m = folium.Map(location=[47.7511, -120.7401], zoom_start=7)

# Add markers for each charging station
for _, row in charging_stations.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Station Name']
    ).add_to(m)

# Display the map
m

## Step 3: Overlay Traffic Congestion Data
In this step, we overlay traffic congestion data onto the existing map of EV charging stations. This helps identify high-demand areas based on traffic flow and congestion levels, which are critical for determining optimal locations for new charging stations.

In [21]:
# Filter locations with traffic congestion level > 1
high_traffic = traffic_congestion[traffic_congestion['Congestion_Level'] > 1]
print("High Traffic Locations:")
print(high_traffic.head())

# Check the number of high-traffic points
print(f"Number of High Traffic Locations: {len(high_traffic)}")

High Traffic Locations:
     FlowDataID  Congestion_Level     Region StationName            Street  \
23         4696                 2  Northwest  002es02430           Rice Rd   
70         5857                 2  Southwest  005es00756             139th   
100        5232                 2  Southwest  005es08132    Mellen Couplet   
135        2533                 3    Olympic  005es10538              14th   
169        5049                 2    Olympic  005es11426  Nisqually I/C-NB   

    Direction   Latitude   Longitude       Timestamp  
23         WB  47.868292 -121.774930  1/3/2025 21:34  
70         SB  45.722828 -122.654920  1/3/2025 21:34  
100        SB  46.706923 -122.973924  1/3/2025 21:34  
135        SB  47.032729 -122.891895  1/3/2025 21:33  
169        SB  47.068113 -122.716559  1/3/2025 21:33  
Number of High Traffic Locations: 147


In [22]:
high_traffic.to_csv('high_traffic_locations.csv', index=False)

## Step 5: Calculate Distances to Existing Charging Stations
We use the Haversine formula to calculate the distance between high-traffic locations and existing charging stations. This allows us to identify high-traffic locations with no charging stations within a specific distance threshold (e.g., 5 km).

In [25]:
from geopy.distance import geodesic

# Define a distance threshold (e.g., 5 km)
distance_threshold = 5

# Ensure high_traffic is a proper copy to avoid SettingWithCopyWarning
high_traffic = high_traffic.copy()

# Function to check if a high-traffic point has a nearby station
def has_nearby_station(lat, lon):
    for _, station in charging_stations.iterrows():
        station_coords = (station['Latitude'], station['Longitude'])
        traffic_coords = (lat, lon)
        # Calculate distance using geodesic (Haversine)
        if geodesic(traffic_coords, station_coords).km <= distance_threshold:
            return True
    return False

# Apply the function to find high-traffic locations with no nearby stations
high_traffic['Has_Station_Nearby'] = high_traffic.apply(
    lambda row: has_nearby_station(row['Latitude'], row['Longitude']), axis=1
)

# Filter locations with no nearby stations
no_station_locations = high_traffic[~high_traffic['Has_Station_Nearby']]

# Check the results
print("High Traffic Locations with No Nearby Stations:")
print(no_station_locations)

# Save the filtered data
no_station_locations.to_csv('no_station_locations.csv', index=False)
print("Exported locations with no nearby stations to 'no_station_locations.csv'.")

High Traffic Locations with No Nearby Stations:
      FlowDataID  Congestion_Level     Region            StationName  \
169         5049                 2    Olympic             005es11426   
187         6022                 3    Olympic             005es12063   
189         6019                 3    Olympic             005es12110   
218          157                 4    Olympic             005es12808   
715         2432                 2  Northwest             005es19469   
716         7380                 3    Olympic             005es19469   
717         6586                 2    Olympic             005es19534   
718         4659                 2  Northwest             005es19534   
719         4660                 2  Northwest             005es19534   
720         6587                 3    Olympic             005es19534   
729         4655                 2  Northwest             005es19735   
730         7571                 2    Olympic             005es19735   
1283        4562

## Step 6: Prioritize Underserved Locations Based on Additional Factors
To refine the list of underserved locations, we analyze congestion levels, cluster locations, and integrate EV ownership data. This helps prioritize locations for new charging stations based on demand and accessibility.

In [26]:
# Analyze congestion levels in underserved locations
congestion_summary = no_station_locations.groupby(['Region', 'Congestion_Level']).size().reset_index(name='Count')
print("Summary of Congestion Levels in Underserved Locations:")
print(congestion_summary)

# Sort by congestion level and count for prioritization
prioritized_locations = no_station_locations.sort_values(by=['Congestion_Level'], ascending=False)
print("Prioritized Locations by Congestion Level:")
print(prioritized_locations.head())

Summary of Congestion Levels in Underserved Locations:
      Region  Congestion_Level  Count
0    Eastern                 2      1
1  Northwest                 2      6
2  Northwest                 3      1
3    Olympic                 2      7
4    Olympic                 3      5
5    Olympic                 4      1
Prioritized Locations by Congestion Level:
     FlowDataID  Congestion_Level   Region StationName  \
218         157                 4  Olympic  005es12808   
189        6019                 3  Olympic  005es12110   
187        6022                 3  Olympic  005es12063   
720        6587                 3  Olympic  005es19534   
716        7380                 3  Olympic  005es19469   

                                         Street Direction   Latitude  \
218                                   S 96th St        NB  47.170303   
189                            JBLM MainGate-NB        SB  47.105594   
187                            JBLM MainGate-SB        SB  47.103080   

In [33]:
from sklearn.cluster import KMeans
import numpy as np
import folium

# Extract latitude and longitude for clustering
coords = no_station_locations[['Latitude', 'Longitude']].values

# Perform k-means clustering
n_clusters = 5  # Adjust based on desired number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
no_station_locations['Cluster'] = kmeans.fit_predict(coords)

# Extract centroids for each cluster
centroids = kmeans.cluster_centers_

# Add cluster assignments to the DataFrame
no_station_locations['Cluster_Latitude'] = no_station_locations['Cluster'].map(
    lambda x: centroids[x][0]
)
no_station_locations['Cluster_Longitude'] = no_station_locations['Cluster'].map(
    lambda x: centroids[x][1]
)

# Save the clustered data to a CSV
clustered_csv = "clustered_no_station_locations.csv"
no_station_locations.to_csv(clustered_csv, index=False)
print(f"Clustered data saved to {clustered_csv}")

# Display cluster summary
cluster_summary = no_station_locations.groupby('Cluster').agg({
    'Latitude': ['min', 'max'],
    'Longitude': ['min', 'max'],
    'Congestion_Level': 'mean'
}).reset_index()

print("Cluster Summary:")
print(cluster_summary)

# Visualize clusters on a map
cluster_map = folium.Map(location=[47.7511, -120.7401], zoom_start=7)  # Centered on Washington
colors = ['red', 'blue', 'green', 'purple', 'orange']

for _, row in no_station_locations.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=6,
        color=colors[row['Cluster']],
        fill=True,
        fill_opacity=0.8,
        popup=(
            f"Cluster: {row['Cluster']}<br>"
            f"Latitude: {row['Latitude']}, Longitude: {row['Longitude']}<br>"
            f"Congestion Level: {row['Congestion_Level']}"
        )
    ).add_to(cluster_map)

# Display cluster centroids on the map
for cluster_idx, centroid in enumerate(centroids):
    folium.Marker(
        location=[centroid[0], centroid[1]],
        popup=f"Cluster {cluster_idx} Centroid",
        icon=folium.Icon(color='black', icon='info-sign')
    ).add_to(cluster_map)

# Save the map
cluster_map.save("clustered_no_station_locations_map.html")
print("Clustered map saved as 'clustered_no_station_locations_map.html'")

Clustered data saved to clustered_no_station_locations.csv
Cluster Summary:
  Cluster   Latitude              Longitude             Congestion_Level
                 min        max         min         max             mean
0       0  47.990358  48.026957 -122.182587 -122.174319         2.250000
1       1  47.673995  47.673995 -117.238818 -117.238818         2.000000
2       2  47.103080  47.170303 -122.593554 -122.472320         3.333333
3       3  47.158600  47.392015 -122.314182 -122.237955         2.250000
4       4  47.068113  47.068113 -122.716559 -122.716559         2.000000
Clustered map saved as 'clustered_no_station_locations_map.html'


C:\Users\Pavan\AppData\Local\Temp\ipykernel_27452\419125513.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_station_locations['Cluster'] = kmeans.fit_predict(coords)
C:\Users\Pavan\AppData\Local\Temp\ipykernel_27452\419125513.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_station_locations['Cluster_Latitude'] = no_station_locations['Cluster'].map(
C:\Users\Pavan\AppData\Local\Temp\ipykernel_27452\419125513.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

Fetch Zipcodes using Reverse Geocoding

In [34]:
#reverse geocode clusterd data
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd

# Initialize geolocator
geolocator = Nominatim(user_agent="ev_station_locator")

# Add rate limiter to prevent excessive requests
reverse_geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)

# Create a DataFrame for storing results
reverse_geocoded_data = []

# Reverse geocode each centroid
for cluster_idx, centroid in enumerate(centroids):
    location = reverse_geocode((centroid[0], centroid[1]))
    if location and 'postcode' in location.raw['address']:
        postal_code = location.raw['address']['postcode']
    else:
        postal_code = None  # If postal code is not found

    reverse_geocoded_data.append({
        'Cluster': cluster_idx,
        'Latitude': centroid[0],
        'Longitude': centroid[1],
        'Postal Code': postal_code
    })

# Convert results to a DataFrame
postal_code_df = pd.DataFrame(reverse_geocoded_data)

# Save to CSV
postal_code_csv = "cluster_centroids_with_postal_codes.csv"
postal_code_df.to_csv(postal_code_csv, index=False)
print(f"Cluster centroids with postal codes saved to {postal_code_csv}")

# Display the results
print(postal_code_df)

Cluster centroids with postal codes saved to cluster_centroids_with_postal_codes.csv
   Cluster   Latitude   Longitude Postal Code
0        0  48.003300 -122.176922       98201
1        1  47.673995 -117.238818       99206
2        2  47.126326 -122.550093       98498
3        3  47.262962 -122.264047       98047
4        4  47.068113 -122.716559       98561


In [35]:
print("Columns in no_station_locations:")
print(no_station_locations.columns)

print("\nColumns in ev_ownership:")
print(ev_ownership.columns)

Columns in no_station_locations:
Index(['FlowDataID', 'Congestion_Level', 'Region', 'StationName', 'Street',
       'Direction', 'Latitude', 'Longitude', 'Timestamp', 'Has_Station_Nearby',
       'Cluster', 'Cluster_Latitude', 'Cluster_Longitude'],
      dtype='object')

Columns in ev_ownership:
Index(['Clean Alternative Fuel Vehicle Type', 'VIN (1-10)', 'DOL Vehicle ID',
       'Model Year', 'Make', 'Model', 'Primary Use', 'Electric Range',
       'Odometer Reading', 'Odometer Reading Description',
       'New or Used Vehicle', 'Sale Price', 'Sale Date', 'Base MSRP',
       'Transaction Type', 'Transaction Date', 'Year', 'County', 'City',
       'State', 'Postal Code',
       '2019 HB 2042: Clean Alternative Fuel Vehicle (CAFV) Eligibility',
       'Meets 2019 HB 2042 Electric Range Requirement',
       'Meets 2019 HB 2042 Sale Date Requirement',
       'Meets 2019 HB 2042 Sale Price/Value Requirement',
       '2019 HB 2042: Battery Range Requirement',
       '2019 HB 2042: Purchase D

## Step 6: Identify Postal Codes for High-Traffic Locations
In this step, we use reverse geocoding to convert the latitude and longitude of high-traffic locations into postal codes. This allows us to merge the high-traffic underserved locations (no_station_locations) with the EV ownership data (ev_ownership) using postal codes as a common key.

Reverse geocoding is performed using the geopy library, which queries OpenStreetMap to fetch postal codes for each location. This step is crucial for associating high-traffic areas with EV ownership trends, enabling further prioritization of underserved regions.

In [48]:
# Reverse Geocoding: Convert Latitude/Longitude to Postal Codes
# This step uses the geopy library to fetch postal codes for each high-traffic location.
# The postal codes will be used to merge with EV ownership data for further analysis.
# Reverse Geocoding: Convert Latitude/Longitude to Postal Codes
# This step uses the geopy library to fetch postal codes for each high-traffic location.
# The postal codes will be used to merge with EV ownership data for further analysis.
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# Initialize the geocoder
geolocator = Nominatim(user_agent="geoapi")

# Function to get postal code from lat/lon
def get_postal_code(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True, timeout=10)
        if location and 'postcode' in location.raw['address']:
            return location.raw['address']['postcode']
    except GeocoderTimedOut:
        return None
    return None

# Add postal codes to no_station_locations
no_station_locations['Postal Code'] = no_station_locations.apply(
    lambda row: get_postal_code(row['Latitude'], row['Longitude']), axis=1
)

# Check the updated DataFrame
print(no_station_locations.head())

     FlowDataID  Congestion_Level     Region StationName            Street  \
169        5049                 2    Olympic  005es11426  Nisqually I/C-NB   
187        6022                 3    Olympic  005es12063  JBLM MainGate-SB   
189        6019                 3    Olympic  005es12110  JBLM MainGate-NB   
218         157                 4    Olympic  005es12808         S 96th St   
715        2432                 2  Northwest  005es19469  MarineView Dr-SB   

    Direction   Latitude   Longitude       Timestamp  Has_Station_Nearby  \
169        SB  47.068113 -122.716559  1/3/2025 21:33               False   
187        SB  47.103080 -122.593554  1/3/2025 21:33               False   
189        SB  47.105594 -122.584404  1/3/2025 21:33               False   
218        NB  47.170303 -122.472320  1/3/2025 21:33               False   
715        NB  47.990387 -122.182573  1/3/2025 21:34               False   

     Cluster  Cluster_Latitude  Cluster_Longitude Postal Code  
169       

In [50]:
print("Column names in no_station_locations:")
print(no_station_locations.columns)

print("\nColumn names in ev_ownership:")
print(ev_ownership.columns)

Column names in no_station_locations:
Index(['FlowDataID', 'Congestion_Level', 'Region', 'StationName', 'Street',
       'Direction', 'Latitude', 'Longitude', 'Timestamp', 'Has_Station_Nearby',
       'Cluster', 'Cluster_Latitude', 'Cluster_Longitude', 'Postal Code'],
      dtype='object')

Column names in ev_ownership:
Index(['Clean Alternative Fuel Vehicle Type', 'VIN (1-10)', 'DOL Vehicle ID',
       'Model Year', 'Make', 'Model', 'Primary Use', 'Electric Range',
       'Odometer Reading', 'Odometer Reading Description',
       'New or Used Vehicle', 'Sale Price', 'Sale Date', 'Base MSRP',
       'Transaction Type', 'Transaction Date', 'Year', 'County', 'City',
       'State', 'Postal Code',
       '2019 HB 2042: Clean Alternative Fuel Vehicle (CAFV) Eligibility',
       'Meets 2019 HB 2042 Electric Range Requirement',
       'Meets 2019 HB 2042 Sale Date Requirement',
       'Meets 2019 HB 2042 Sale Price/Value Requirement',
       '2019 HB 2042: Battery Range Requirement',
       

In [51]:
print("Column names in no_station_locations:", list(no_station_locations.columns))
print("\nFirst few rows of no_station_locations:")
print(no_station_locations.head())

# Explicitly check if 'Postal Code' is in the columns
if 'Postal Code' in no_station_locations.columns:
    print("\n'Postal Code' exists in no_station_locations.")
else:
    print("\n'Postal Code' does NOT exist in no_station_locations.")

Column names in no_station_locations: ['FlowDataID', 'Congestion_Level', 'Region', 'StationName', 'Street', 'Direction', 'Latitude', 'Longitude', 'Timestamp', 'Has_Station_Nearby', 'Cluster', 'Cluster_Latitude', 'Cluster_Longitude', 'Postal Code']

First few rows of no_station_locations:
     FlowDataID  Congestion_Level     Region StationName            Street  \
169        5049                 2    Olympic  005es11426  Nisqually I/C-NB   
187        6022                 3    Olympic  005es12063  JBLM MainGate-SB   
189        6019                 3    Olympic  005es12110  JBLM MainGate-NB   
218         157                 4    Olympic  005es12808         S 96th St   
715        2432                 2  Northwest  005es19469  MarineView Dr-SB   

    Direction   Latitude   Longitude       Timestamp  Has_Station_Nearby  \
169        SB  47.068113 -122.716559  1/3/2025 21:33               False   
187        SB  47.103080 -122.593554  1/3/2025 21:33               False   
189        SB 

In [52]:
print("\nNull values in no_station_locations['Postal Code']:", no_station_locations['Postal Code'].isnull().sum())
print("Null values in ev_ownership['Postal Code']:", ev_ownership['Postal Code'].isnull().sum())


Null values in no_station_locations['Postal Code']: 0
Null values in ev_ownership['Postal Code']: 89


In [53]:
ev_ownership['Postal Code'].fillna('Unknown', inplace=True)

C:\Users\Pavan\AppData\Local\Temp\ipykernel_27452\836039074.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ev_ownership['Postal Code'].fillna('Unknown', inplace=True)
C:\Users\Pavan\AppData\Local\Temp\ipykernel_27452\836039074.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ev_ownership['Postal Code'].fillna('Unknown', inplace=True)


In [54]:
ev_ownership['Postal Code'] = ev_ownership['Postal Code'].astype(str).fillna('Unknown')

In [58]:
# Clean column names using string.replace to handle \xa0
ev_ownership.columns = [col.replace('\xa0', ' ') for col in ev_ownership.columns]

# Verify the cleaned column names
print("Cleaned Columns in ev_ownership:", ev_ownership.columns)

Cleaned Columns in ev_ownership: Index(['Clean Alternative Fuel Vehicle Type', 'VIN (1-10)', 'DOL Vehicle ID',
       'Model Year', 'Make', 'Model', 'Primary Use', 'Electric Range',
       'Odometer Reading', 'Odometer Reading Description',
       'New or Used Vehicle', 'Sale Price', 'Sale Date', 'Base MSRP',
       'Transaction Type', 'Transaction Date', 'Year', 'County', 'City',
       'State', 'Postal Code',
       '2019 HB 2042: Clean Alternative Fuel Vehicle (CAFV) Eligibility',
       'Meets 2019 HB 2042 Electric Range Requirement',
       'Meets 2019 HB 2042 Sale Date Requirement',
       'Meets 2019 HB 2042 Sale Price/Value Requirement',
       '2019 HB 2042: Battery Range Requirement',
       '2019 HB 2042: Purchase Date Requirement',
       '2019 HB 2042: Sale Price/Value Requirement',
       'Electric Vehicle Fee Paid', 'Transportation Electrification Fee Paid',
       'Hybrid Vehicle Electrification Fee Paid', '2020 GEOID',
       'Legislative District', 'Electric Utility']

In [60]:
# Remove .0 from Postal Codes in ev_ownership
ev_ownership['Postal Code'] = ev_ownership['Postal Code'].str.replace('.0', '', regex=False)

# Strip leading/trailing whitespace
ev_ownership['Postal Code'] = ev_ownership['Postal Code'].str.strip()

In [62]:
no_station_locations['Postal Code'] = no_station_locations['Postal Code'].astype(str)

In [63]:
# Ensure Postal Code columns are strings
print("Cleaned Postal Codes in no_station_locations:")
print(no_station_locations['Postal Code'].unique())

print("\nCleaned Postal Codes in ev_ownership:")
print(ev_ownership['Postal Code'].unique())

Cleaned Postal Codes in no_station_locations:
['98561' '98433' '98499' '98201' '98270' '99206' '98047' '98390' '98373']

Cleaned Postal Codes in ev_ownership:
['98052' '98125' '98177' ... '33432' '78413' '90262']


In [64]:
# Merge using cleaned postal codes
merged_data = no_station_locations.merge(ev_ownership, how='left', on='Postal Code')

# Check the results
print("Merged Data Sample:")
print(merged_data.head())

Merged Data Sample:
   FlowDataID  Congestion_Level   Region StationName            Street  \
0        5049                 2  Olympic  005es11426  Nisqually I/C-NB   
1        6022                 3  Olympic  005es12063  JBLM MainGate-SB   
2        6022                 3  Olympic  005es12063  JBLM MainGate-SB   
3        6022                 3  Olympic  005es12063  JBLM MainGate-SB   
4        6022                 3  Olympic  005es12063  JBLM MainGate-SB   

  Direction   Latitude   Longitude       Timestamp  Has_Station_Nearby  ...  \
0        SB  47.068113 -122.716559  1/3/2025 21:33               False  ...   
1        SB  47.103080 -122.593554  1/3/2025 21:33               False  ...   
2        SB  47.103080 -122.593554  1/3/2025 21:33               False  ...   
3        SB  47.103080 -122.593554  1/3/2025 21:33               False  ...   
4        SB  47.103080 -122.593554  1/3/2025 21:33               False  ...   

   Meets 2019 HB 2042 Sale Price/Value Requirement  \
0     

In [65]:
unmatched = merged_data[merged_data['VIN (1-10)'].isnull()]
print(f"Number of unmatched postal codes: {len(unmatched)}")
print("Unmatched postal codes:")
print(unmatched['Postal Code'].unique())

Number of unmatched postal codes: 1
Unmatched postal codes:
['98561']


In [66]:
# Count matched postal codes
matched = merged_data[~merged_data['VIN (1-10)'].isnull()]
matched_postal_codes = matched['Postal Code'].nunique()  # Count unique matched postal codes

print(f"Number of matched postal codes: {matched_postal_codes}")

Number of matched postal codes: 8


In [67]:
merged_data.to_csv('merged_no_station_ev_data.csv', index=False)
print("Exported merged data to 'merged_no_station_ev_data.csv'.")

Exported merged data to 'merged_no_station_ev_data.csv'.


We will prioritize underserved high-traffic locations based on:

Traffic Congestion Levels: Higher congestion levels indicate higher potential demand.
EV Ownership Trends: Matched postal codes with significant EV ownership.

In [69]:
# Aggregate EV ownership counts per postal code
merged_data['EV_Count'] = merged_data.groupby('Postal Code')['VIN (1-10)'].transform('count')

# Fill missing EV counts with 0 (for unmatched postal codes)
merged_data['EV_Count'].fillna(0,inplace=True)

C:\Users\Pavan\AppData\Local\Temp\ipykernel_27452\1548522480.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_data['EV_Count'].fillna(0,inplace=True)


In [70]:
# Prioritize locations
prioritized_data = merged_data.sort_values(by=['Congestion_Level', 'EV_Count'], ascending=[False, False])

# Display top prioritized locations
print("Top Prioritized Locations for Charging Stations:")
print(prioritized_data[['Postal Code', 'Congestion_Level', 'EV_Count', 'Latitude', 'Longitude']].head(10))

# Save the prioritized data
prioritized_data.to_csv('prioritized_new_locations.csv', index=False)
print("Exported prioritized locations to 'prioritized_new_locations.csv'.")

Top Prioritized Locations for Charging Stations:
     Postal Code  Congestion_Level  EV_Count   Latitude  Longitude
1463       98499                 4      1834  47.170303 -122.47232
1464       98499                 4      1834  47.170303 -122.47232
1465       98499                 4      1834  47.170303 -122.47232
1466       98499                 4      1834  47.170303 -122.47232
1467       98499                 4      1834  47.170303 -122.47232
1468       98499                 4      1834  47.170303 -122.47232
1469       98499                 4      1834  47.170303 -122.47232
1470       98499                 4      1834  47.170303 -122.47232
1471       98499                 4      1834  47.170303 -122.47232
1472       98499                 4      1834  47.170303 -122.47232
Exported prioritized locations to 'prioritized_new_locations.csv'.


we can aggregate the results for each postal code to avoid repetitive rows. This will combine data for each postal code into a single entry with relevant statistics like maximum congestion level, total EV count, and average or representative latitude/longitude.

In [71]:
# Aggregate data by postal code
final_aggregated_data = prioritized_data.groupby('Postal Code').agg({
    'Congestion_Level': 'max',   # Maximum congestion level
    'EV_Count': 'sum',           # Total EV count
    'Latitude': 'mean',          # Average latitude
    'Longitude': 'mean'          # Average longitude
}).reset_index()

# Remove rows where EV_Count is 0
final_aggregated_data = final_aggregated_data[final_aggregated_data['EV_Count'] > 0]

# Display the aggregated data
print("Aggregated Data by Postal Code:")
print(final_aggregated_data.head())

# Save the aggregated data
final_aggregated_data.to_csv('aggregated_prioritized_locations.csv', index=False)
print("Exported aggregated prioritized locations to 'aggregated_prioritized_locations.csv'.")

Aggregated Data by Postal Code:
  Postal Code  Congestion_Level   EV_Count   Latitude   Longitude
0       98047                 3    2277081  47.258896 -122.259882
1       98201                 3  292410000  47.995436 -122.177142
2       98270                 2  121881600  48.026890 -122.176260
3       98373                 2    5125696  47.158600 -122.314182
4       98390                 3   27751824  47.292102 -122.254637
Exported aggregated prioritized locations to 'aggregated_prioritized_locations.csv'.


We merge the aggregated_prioritized_locations.csv with cluster_centroids_with_postal_codes.csv using an outer join. This step ensures that postal codes from the clustered centroids are added to the prioritized locations for final mapping.

In [73]:
import pandas as pd
import folium

# Load the datasets
aggregated_prioritized_locations = pd.read_csv("aggregated_prioritized_locations.csv")
cluster_centroids_with_postal_codes = pd.read_csv("cluster_centroids_with_postal_codes.csv")

# Perform an outer join on the postal codes
final_data = pd.merge(
    aggregated_prioritized_locations,
    cluster_centroids_with_postal_codes,
    on="Postal Code",
    how="outer",
    suffixes=('_aggregated', '_clustered')
)

# Check the merged dataset
print("Final Merged Data:")
print(final_data.head())

# Save the final dataset to a CSV
final_csv = "final_prioritized_locations.csv"
final_data.to_csv(final_csv, index=False)
print(f"Final merged data saved to {final_csv}")

# Generate the final HTML map
final_map = folium.Map(location=[47.7511, -120.7401], zoom_start=7)  # Centered on Washington

# Add markers for locations with postal codes
for _, row in final_data.dropna(subset=["Postal Code"]).iterrows():
    # Use the aggregated lat/lon if available, otherwise use the clustered lat/lon
    latitude = row['Latitude_aggregated'] if not pd.isna(row['Latitude_aggregated']) else row['Latitude_clustered']
    longitude = row['Longitude_aggregated'] if not pd.isna(row['Longitude_aggregated']) else row['Longitude_clustered']

    folium.Marker(
        location=[latitude, longitude],
        popup=f"Postal Code: {row['Postal Code']}<br>Cluster: {row.get('Cluster', 'N/A')}",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(final_map)

# Save the final map
final_map.save("final_prioritized_locations_map.html")
print("Final HTML map saved as 'final_prioritized_locations_map.html'")

Final Merged Data:
   Postal Code  Congestion_Level     EV_Count  Latitude_aggregated  \
0        98047               3.0    2277081.0            47.258896   
1        98201               3.0  292410000.0            47.995436   
2        98270               2.0  121881600.0            48.026890   
3        98373               2.0    5125696.0            47.158600   
4        98390               3.0   27751824.0            47.292102   

   Longitude_aggregated  Cluster  Latitude_clustered  Longitude_clustered  
0           -122.259882      3.0           47.262962          -122.264047  
1           -122.177142      0.0           48.003300          -122.176922  
2           -122.176260      NaN                 NaN                  NaN  
3           -122.314182      NaN                 NaN                  NaN  
4           -122.254637      NaN                 NaN                  NaN  
Final merged data saved to final_prioritized_locations.csv
Final HTML map saved as 'final_prioritized_lo

In [74]:
import folium
from folium.plugins import HeatMap

# Prepare data for heatmap (EV ownership and underserved areas)
ev_data = aggregated_prioritized_locations[['Latitude', 'Longitude', 'EV_Count']].dropna()
underserved_data = no_station_locations[['Latitude', 'Longitude']].dropna()

# Create a base map
heatmap = folium.Map(location=[47.7511, -120.7401], zoom_start=7)

# Add EV ownership heatmap
HeatMap(data=ev_data[['Latitude', 'Longitude', 'EV_Count']].values, radius=15, max_zoom=10).add_to(heatmap)

# Add underserved areas as markers
for _, row in underserved_data.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=6,
        color='red',
        fill=True,
        fill_opacity=0.8,
        popup="Underserved Area"
    ).add_to(heatmap)

# Save the map
heatmap.save("heatmap_ev_ownership_and_underserved.html")
print("Heatmap saved as 'heatmap_ev_ownership_and_underserved.html'")

Heatmap saved as 'heatmap_ev_ownership_and_underserved.html'
